Les Imports

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding,LSTM,Dense,Bidirectional
from tensorflow.keras import Sequential


Data preparation

In [2]:
# Load the Shakespeare dataset
data = open("t8.shakespeare.txt").read()

# Tokenize the data
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
sequence_data = tokenizer.texts_to_sequences([data])[0]

# Generate training sequences
sequences = []
for i in range(1, len(sequence_data)):
    sequence = sequence_data[i-1:i+1]
    sequences.append(sequence)

# Pad the sequences
max_sequence_len = max([len(x) for x in sequences])
sequences = pad_sequences(sequences, maxlen=max_sequence_len, padding='pre')

# Split the data into training and validation sets
train_size = int(len(sequences) * 0.8)
train_sequences = sequences[:train_size]
val_sequences = sequences[train_size:]

Model 

In [3]:
# Define the RNN+LSTM language model
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64, input_length=max_sequence_len-1),
    Bidirectional(LSTM(32)),
    Dense(len(tokenizer.word_index)+1, activation='softmax')
])

Compile // Train

In [4]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
# Train the model
epochs = 50
for epoch in range(epochs):
    print("Epoch {}/{}".format(epoch+1, epochs))
    model.fit(train_sequences[:,:-1], train_sequences[:,-1], epochs=1, validation_data=(val_sequences[:,:-1], val_sequences[:,-1]))
    model.reset_states()

Epoch 1/50
22673/22673 [==============================] - 552s 24ms/step - loss: 6.5856 - accuracy: 0.0738 - val_loss: 6.8047 - val_accuracy: 0.0892
Epoch 2/50
22673/22673 [==============================] - 548s 24ms/step - loss: 6.1829 - accuracy: 0.0995 - val_loss: 6.7935 - val_accuracy: 0.0933
Epoch 3/50
22673/22673 [==============================] - 556s 25ms/step - loss: 6.0672 - accuracy: 0.1057 - val_loss: 6.8121 - val_accuracy: 0.0930
Epoch 4/50
22673/22673 [==============================] - 592s 26ms/step - loss: 5.9802 - accuracy: 0.1107 - val_loss: 6.8469 - val_accuracy: 0.0921
Epoch 5/50
22673/22673 [==============================] - 561s 25ms/step - loss: 5.8990 - accuracy: 0.1144 - val_loss: 6.9473 - val_accuracy: 0.0930
Epoch 6/50
22673/22673 [==============================] - 552s 24ms/step - loss: 5.8260 - accuracy: 0.1182 - val_loss: 7.0413 - val_accuracy: 0.0920
Epoch 7/50
22673/22673 [==============================] - 549s 24ms/step - loss: 5.7724 - accuracy: 0.1209

KeyboardInterrupt: 

In [9]:
# Generate new text using the trained model
seed_text = "Juliet"
next_words = 100
for _ in range(next_words):
    seed_sequence = tokenizer.texts_to_sequences([seed_text])[0]
    padded_sequence = pad_sequences([seed_sequence], maxlen=max_sequence_len-1, padding='pre')
    predicted_index = model.predict(padded_sequence, verbose=0)
    predicted_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            predicted_word = word
            break
    seed_text += " " + predicted_word

print(seed_text)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()